In [46]:
using ReverseDiff
using ProgressMeter
using Base.Test
using MLDataPattern
using Plots
gr()

Plots.GRBackend()

In [93]:
module Nets

import ReverseDiff
using MLDataPattern: batchview, shuffleobs

head(t::Tuple) = tuple(t[1])

function viewblocks{T <: NTuple}(data::AbstractArray, shapes::AbstractVector{T})
    starts = cumsum(vcat([1], prod.(shapes)))
    [reshape(view(data, starts[i]:(starts[i+1] - 1)), shapes[i]) for i in 1:length(shapes)]
end

type PANet{Sensitive}
    shapes::Vector{NTuple{2, Int}}
end

nweights(net::PANet) = sum(prod, net.shapes)
nbiases(net::PANet) = sum(first, net.shapes)
nparams(net::PANet) = nweights(net) + nbiases(net)
Base.rand(net::PANet) = rand(nparams(net))
Base.randn(net::PANet) = randn(nparams(net))

function predict(net::PANet{true}, params::AbstractVector, x::AbstractVector)
    weights = viewblocks(params, net.shapes)
    biases = viewblocks(@view(params[(nweights(net) + 1):end]), head.(net.shapes))
    y = similar(x, Base.promote_eltype(params, x), (length(x), 1))
    y .= x
    J = eye(eltype(y), length(x))
    for i in 1:(length(net.shapes) - 1)
        w = weights[i]
        y = w * y .+ biases[i]
        J = w * J
        z = [yy >= 0 ? 1.0 : 0.1 for yy in y]
        y = y .* z
        J = J .* z
    end
    w = weights[end]
    vcat(vec(w * y), vec(w * J))
end

function predict(net::PANet{false}, params::AbstractVector, x::AbstractVector)
    weights = viewblocks(params, net.shapes)
    biases = viewblocks(@view(params[(nweights(net) + 1):end]), head.(net.shapes))
    y = similar(x, Base.promote_eltype(params, x), (length(x), 1))
    y .= x
    for i in 1:(length(net.shapes) - 1)
        w = weights[i]
        y = w * y .+ biases[i]
        z = [yy >= 0 ? 1.0 : 0.1 for yy in y]
        y = y .* z
    end
    w = weights[end]
    vec(w * y)
end

function sgd!(loss, params, data, lr=0.01, momentum=0.8, batchsize=1)
    last_descent = zeros(params)
    dw = zeros(params)
    dw_sample = zeros(dw)
    for batch in batchview(shuffleobs(data), batchsize)
        dw .= 0
        for (x, y) in batch
            ReverseDiff.gradient!(dw_sample, w -> loss(w, x, y), params)
            if any(isnan.(dw_sample))
                @show params x y dw_sample
                error("nan")
            end
            dw .+= dw_sample
        end
        for i in eachindex(params)
            v = lr * dw[i] + momentum * last_descent[i]
            params[i] -= v
            last_descent[i] = v
        end
    end
    params
end

end

Nets

In [94]:
data = [
begin
    x = randn()
    if x >= 0
        [x], [x, 1]
    else
        [x], [-x, -1]
    end
    end for i in 1:20];
train_data, test_data = splitobs(data, 0.5)


shapes = [(4, 1), (4, 4), (1, 4)]
net = Nets.PANet{true}(shapes)
params = randn(net)
@inferred Nets.predict(net, params, [0])
predict = (params, x) -> Nets.predict(net, params, x)
loss = (params, x, y) -> sumabs2(y - predict(params, x))

(::#83) (generic function with 1 method)

In [96]:
@showprogress for i in 1:10
    Nets.sgd!(loss, params, train_data)
end

xx = linspace(-2, 2)
plt = plot(xx, x -> predict(params, [x])[1], ylim=(-1, 1), legend=nothing, linewidth=4)
plot!(plt, [x for (x, y) in train_data], [y for (x, y) in train_data], 
line=nothing,
markershape=:circle,
markersize=1
)
plt

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:3.2; stroke-opacity:1; fill:none" points="
 63.0534,-185.908 72.9649,-170.311 82.8765,-154.715 92.7881,-139.118 102.7,-123.522 112.611,-107.925 122.523,-92.3286 132.435,-76.732 142.346,-61.1355 152.258,-45.5389 
 162.169,-29.9424 172.081,-14.3458 181.992,1.25075 191.904,16.8473 201.816,32.4439 211.727,48.0404 221.639,63.637 231.55,79.2335 241.462,89.6556 251.374,105.106 
 261.285,120.556 271.197,136.006 281.108,151.179 291.02,165.815 300.932,180.451 310.843,187.152 320.755,171.004 330.666,154.856 340.578,138.709 350.49,122.996 
 360.401,107.539 370.313,92.0829 380.224,76.6263 390.136,61.1697 400.048,45.7131 409.959,30.2565 419.871,14.8 429.782,-0.65662 439.694,-16.1132 449.606,-31.5698 
 459.517,-47.0264 469.429,-62.483 479.34,-77.9395 489.252,-93.3961 499.163,-108.853 509.075,-124.309 518.987,-139.766 528.898,-155.222 538.81,-170.679 548.721,-186.136 
 
 "/>

In [28]:
Nets.predict(Nets.PANet{false}(shapes), params, [0])

1-element Array{Float64,1}:
 1.18915

In [ ]:


function sgd(net, params, data, lr=0.1, momentum=0.8, batchsize=1)
    last_descent = zeros(w)
    for batch in batchview(shuffleobs(data), batchsize)
        dw = zeros(w)
        for (x, y) in batch
            sample_grad = ReverseDiff.gradient(w -> loss(w, shapes, x, y), w)
            dw .+= sample_grad
        end
        for i in 1:length(w)
            v = lr * dw[i] + momentum * last_descent[i]
            w[i] -= v
            last_descent[i] = v
        end
    end
    w
end

function train(w, shapes, data, lr=0.1, momentum=0.8)
    last_descent = zeros(w)
    for batch in batchview(data, 1)
        dw = zeros(w)
        for (x, y) in batch
            sample_grad = ReverseDiff.gradient(w -> loss(w, shapes, x, y), w)
            dw .+= sample_grad
        end
        for i in 1:length(w)
            v = lr * dw[i] + momentum * last_descent[i]
            w[i] -= v
            last_descent[i] = v
        end
    end
    w
end

w = rand(sum(prod, shapes) + sum(first, shapes))

@showprogress for i in 1:100
    train(w, shapes, train_data, 0.01, 0.5)
end

xx = linspace(-2, 2)
plt = plot(xx, x -> predict(w, shapes, [x])[1], ylim=(-1, 1), legend=nothing, linewidth=4)
plot!(plt, [x for (x, y) in train_data], [y for (x, y) in train_data], 
line=nothing,
markershape=:circle,
markersize=1
)
plt

In [148]:
predict = (flat_weights, shapes, x) -> begin
    weights = viewblocks(flat_weights, shapes)
    num_weights = sum(length, weights)
    biases = viewblocks(@view(flat_weights[(num_weights + 1):end]), head.(shapes))
    y = x
    J = eye(length(x))
    for i in 1:(length(shapes) - 1)
        w = weights[i]
        y = w * y .+ biases[i]
        J = w * J
        z = [yy >= 0 ? 1.0 : 0.1 for yy in y]
        y = y .* z
        J = J .* z
    end
    w = weights[end]
    vcat(vec(w * y), vec(w * J))
end

# Include gradients in loss function
loss = (w, shapes, x, y) -> sumabs2(y - predict(w, shapes, x))

function train(w, shapes, data, lr=0.1, momentum=0.8)
    last_descent = zeros(w)
    for batch in batchview(data, 1)
        dw = zeros(w)
        for (x, y) in batch
            sample_grad = ReverseDiff.gradient(w -> loss(w, shapes, x, y), w)
            dw .+= sample_grad
        end
        for i in 1:length(w)
            v = lr * dw[i] + momentum * last_descent[i]
            w[i] -= v
            last_descent[i] = v
        end
    end
    w
end

w = rand(sum(prod, shapes) + sum(first, shapes))

@showprogress for i in 1:100
    train(w, shapes, train_data, 0.01, 0.5)
end

xx = linspace(-2, 2)
plt = plot(xx, x -> predict(w, shapes, [x])[1], ylim=(-1, 1), legend=nothing, linewidth=4)
plot!(plt, [x for (x, y) in train_data], [y for (x, y) in train_data], 
line=nothing,
markershape=:circle,
markersize=1
)
plt

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:3.2; stroke-opacity:1; fill:none" points="
 55.2446,-216.289 64.2427,-196.372 73.2408,-176.455 82.2389,-156.538 91.237,-136.621 100.235,-116.704 109.233,-96.7875 118.231,-77.0984 127.229,-60.1664 136.227,-43.7938 
 145.226,-28.5546 154.224,-13.3153 163.222,1.92389 172.22,17.1631 181.218,32.4024 190.216,47.6416 199.214,62.8808 208.212,78.12 217.21,93.3593 226.208,108.599 
 235.207,123.838 244.205,139.077 253.203,152.196 262.201,156.029 271.199,156.512 280.197,156.995 289.195,157.478 298.193,151.498 307.191,139.259 316.19,127.021 
 325.188,114.782 334.186,102.45 343.184,87.6768 352.182,72.9035 361.18,58.1302 370.178,43.3569 379.176,27.8331 388.174,12.0211 397.172,-3.79099 406.171,-19.603 
 415.169,-35.4151 424.167,-51.2272 433.165,-67.0392 442.163,-82.8513 451.161,-98.6633 460.159,-114.475 469.157,-130.287 478.155,-146.099 487.153,-161.912 496.152,-177.724 
 
 "/>

In [151]:
predict = (flat_weights, shapes, x) -> begin
    weights = viewblocks(flat_weights, shapes)
    num_weights = sum(length, weights)
    biases = viewblocks(@view(flat_weights[(num_weights + 1):end]), head.(shapes))
    y = x
    J = eye(length(x))
    for i in 1:(length(shapes) - 1)
        w = weights[i]
        y = w * y .+ biases[i]
        J = w * J
        z = [yy >= 0 ? 1.0 : 0.1 for yy in y]
        y = y .* z
        J = J .* z
    end
    w = weights[end]
    vcat(vec(w * y), vec(w * J))
end

# Loss function with no gradients
loss = (w, shapes, x, y) -> sumabs2(y[1] - predict(w, shapes, x)[1])

function train(w, shapes, data, lr=0.1, momentum=0.8)
    last_descent = zeros(w)
    for batch in batchview(data, 1)
        dw = zeros(w)
        for (x, y) in batch
            sample_grad = ReverseDiff.gradient(w -> loss(w, shapes, x, y), w)
            dw .+= sample_grad
        end
        for i in 1:length(w)
            v = lr * dw[i] + momentum * last_descent[i]
            w[i] -= v
            last_descent[i] = v
        end
    end
    w
end

w = rand(sum(prod, shapes) + sum(first, shapes))

@showprogress for i in 1:100
    train(w, shapes, train_data, 0.01, 0.5)
end

xx = linspace(-2, 2)
plt = plot(xx, x -> predict(w, shapes, [x])[1], ylim=(-1, 1), legend=nothing, linewidth=4)
plot!(plt, [x for (x, y) in train_data], [y for (x, y) in train_data], 
line=nothing,
markershape=:circle,
markersize=1
)
plt

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:3.2; stroke-opacity:1; fill:none" points="
 55.2446,187.529 65.9602,186.112 76.6758,184.695 87.3915,183.278 98.1071,181.861 108.823,180.444 119.538,179.027 130.254,177.611 140.97,176.194 151.685,174.777 
 162.401,173.36 173.117,171.907 183.832,170.298 194.548,168.085 205.264,165.258 215.979,162.431 226.695,159.604 237.41,156.777 248.126,153.95 258.842,151.123 
 269.557,148.296 280.273,145.468 290.989,142.641 301.704,139.814 312.42,136.987 323.136,134.16 333.851,126.99 344.567,117.503 355.283,108.016 365.998,98.5295 
 376.714,89.0427 387.429,79.5559 398.145,70.0691 408.861,60.5823 419.576,51.0955 430.292,41.6087 441.008,32.1218 451.723,22.635 462.439,13.1482 473.155,3.66144 
 483.87,-5.82536 494.586,-15.3122 505.302,-24.799 516.017,-34.2858 526.733,-43.7726 537.448,-53.2594 548.164,-62.7462 558.88,-72.233 569.595,-81.7198 580.311,-91.2066 
 
 "/>